In [39]:
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob
from scipy.stats import pearsonr
import sys
import os

In [40]:
# Add parent directory to path
sys.path.append(os.path.abspath(".."))
from scripts.news_utils import (
    load_news_data,
    parse_dates,
    filter_by_ticker
)

ticker = 'GOOG'
news_path = '../data/raw_analyst_ratings.csv'
stock_path = f'../data/yfinance_data/{ticker}_historical_data.csv'

In [41]:
# ------------------ Load & Preprocess News Data ------------------
df_news = load_news_data(news_path)
df_news = parse_dates(df_news)
df_news_goog = filter_by_ticker(df_news, ticker).copy()


In [42]:
# Remove timezone for alignment
df_news_goog = df_news_goog.copy()
df_news_goog['date'] = df_news_goog['date'].dt.tz_localize(None)

print("\nGOOG News Data Info:")
print(df_news_goog.info())


GOOG News Data Info:
<class 'pandas.core.frame.DataFrame'>
Index: 1199 entries, 565380 to 566578
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   headline   1199 non-null   object        
 1   url        1199 non-null   object        
 2   publisher  1199 non-null   object        
 3   date       1199 non-null   datetime64[ns]
 4   stock      1199 non-null   object        
 5   date_only  1199 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 65.6+ KB
None


In [43]:
# ------------------  Sentiment Analysis ------------------
def get_sentiment(headline):
    return TextBlob(str(headline)).sentiment.polarity  # [-1.0, +1.0]

df_news_goog.loc[:, 'sentiment'] = df_news_goog['headline'].apply(get_sentiment)

In [44]:
# Aggregate and convert index to datetime
daily_sentiment = (
    df_news_goog
      .groupby('date_only')['sentiment']
      .mean()
      .reset_index(name='mean_polarity')
)

# Convert 'date_only' to datetime and set as index
daily_sentiment['date_only'] = pd.to_datetime(daily_sentiment['date_only'])
daily_sentiment = daily_sentiment.set_index('date_only').sort_index()

print(f"\n{ticker} Daily Sentiment (First 5 Rows):")
print(daily_sentiment.head())



GOOG Daily Sentiment (First 5 Rows):
            mean_polarity
date_only                
2018-11-13       0.000000
2018-11-14       0.056818
2018-11-15       0.109028
2018-11-16      -0.190000
2018-11-17       0.500000


In [36]:
# ------------------  Load Stock Data ------------------
df_stock = pd.read_csv(stock_path)
df_stock['Date'] = pd.to_datetime(df_stock['Date'])
df_stock = df_stock.set_index('Date').sort_index()

# Align stock start date with earliest news date
earliest_news_date = df_news_goog['date'].min().normalize()
df_stock = df_stock[df_stock.index >= earliest_news_date]

# Compute daily returns
df_stock['Daily_Return'] = df_stock['Close'].pct_change() * 100

print("\nGOOG Stock Data Info:")
print(df_stock.info())



GOOG Stock Data Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1435 entries, 2018-11-13 to 2024-07-30
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1435 non-null   float64
 1   High          1435 non-null   float64
 2   Low           1435 non-null   float64
 3   Close         1435 non-null   float64
 4   Adj Close     1435 non-null   float64
 5   Volume        1435 non-null   int64  
 6   Dividends     1435 non-null   float64
 7   Stock Splits  1435 non-null   float64
 8   Daily_Return  1434 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 112.1 KB
None


In [48]:
# ------------------ Step 4: Correlation Analysis ------------------
# Join stock returns with daily sentiment (inner join keeps only dates that appear in both)
df_merged = (
    df_stock[['Daily_Return']]
    .join(daily_sentiment.rename(columns={'mean_polarity':'sentiment'}), how='inner')
    .dropna(subset=['Daily_Return', 'sentiment'])
)

if len(df_merged) > 1:
    correlation, p_value = pearsonr(df_merged['sentiment'], df_merged['Daily_Return'])
    print(f"\nPearson Correlation for {ticker}: {correlation:.4f} (p-value: {p_value:.4f})")
else:
    print(f"\nInsufficient data for correlation in {ticker}")


Pearson Correlation for GOOG: 0.0567 (p-value: 0.2898)


In [49]:
# 1. Plot Daily Mean Sentiment
plt.figure(figsize=(12, 6))
daily_sentiment['mean_polarity'].plot(color='purple')
plt.title(f'{ticker} Daily News Sentiment (Mean Polarity)')
plt.xlabel('Date')
plt.ylabel('Sentiment Polarity (âˆ’1 to +1)')
plt.tight_layout()
plt.show()

# 2. Plot Sentiment vs. Daily Return Scatter
plt.figure(figsize=(10, 6))
plt.scatter(df_merged['sentiment'], df_merged['Daily_Return'], alpha=0.5, color='teal')
plt.title(f'{ticker} Daily Sentiment vs. Daily Returns')
plt.xlabel('Mean Daily Polarity')
plt.ylabel('Daily Return (%)')
plt.axhline(0, color='gray', linestyle='--')
plt.axvline(0, color='gray', linestyle='--')
plt.tight_layout()
plt.show()